In [1]:
import os
import pandas as pd

HORNSGATAN_HOME = os.environ["HORNSGATAN_HOME"]
os.chdir(HORNSGATAN_HOME)
print("Current directory:", os.getcwd())

from src.tools import mytools


date = "2020-01-01"
detector = "e2w_out"
path  = "data/daily_splitted_data/"


Current directory: /home/kaveh/projects/Hornsgatan


In [2]:
data = pd.read_csv(f"{path}data_{date}.csv")
data.head()

,detector_id,time_detector_real,speed_detector_real,day,date
0,w2e_in,1577836811,33,Wednesday,2020-01-01
1,w2e_in,1577836814,27,Wednesday,2020-01-01
2,w2e_in,1577836815,32,Wednesday,2020-01-01
3,e2w_in,1577836818,32,Wednesday,2020-01-01
4,e2w_in,1577836820,27,Wednesday,2020-01-01


In [3]:
data = data[data["detector_id"]==detector]
data.head()

,detector_id,time_detector_real,speed_detector_real,day,date
12,e2w_out,1577836850,28,Wednesday,2020-01-01
17,e2w_out,1577836881,40,Wednesday,2020-01-01
29,e2w_out,1577836917,13,Wednesday,2020-01-01
41,e2w_out,1577836953,33,Wednesday,2020-01-01
45,e2w_out,1577836960,35,Wednesday,2020-01-01


In [4]:
len(data)

921

In [5]:
from src.tools import mytools


In [6]:
#mytools.delete_all_except_list(f"data/sim_intermediate_data/", [".gitkeep"])